<a href="https://colab.research.google.com/github/tjbarnum13/chm352/blob/main/PIB_hfscf_interactive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# for local installation
# sys.path.append("../") # go to parent dir if running notebook directly in sub-folder

# for Google Colab
!git clone http://github.com/tjbarnum13/Fourier-DVR-1D.git
sys.path.insert(0,'/content/Fourier-DVR-1D')
from fourier_DVR_1D import Domain_Fourier_DVR_1D

fatal: destination path 'Fourier-DVR-1D' already exists and is not an empty directory.


In [2]:
# settings
m = 1.0 # particle mass
n_DVR = 300 # number of DVR grid points
n_g = 1001 # number of grid points for plotting
n_plot = 5 # number of eigenstates to plot
scale = 1.0 # used to adjust size of wavefunctions on plot

pib_length = 1 # box size
well_depth = 1e7 # depth of the finite well
pib_lowerlimit=-pib_length/2
pib_upperlimit=pib_length/2
x_min = 2*pib_lowerlimit
x_max = 2*pib_upperlimit

# set up grid of points for the two particles
x1 = np.linspace(x_min,x_max,n_g)
x2 = np.linspace(x_min,x_max,n_g)
domain = Domain_Fourier_DVR_1D(x_min, x_max, n_DVR)


In [7]:
def run(n1, n2, a, cutoff):

  # inital wave function guesses are just the particle in a box!
  def psi_initial(x,n):
    if n%2 == 0:
      y = np.piecewise(
            x,
            [x < pib_lowerlimit, (x >= pib_lowerlimit) & (x <= pib_upperlimit), x > pib_upperlimit],
            [0, lambda x: np.sqrt(2/pib_length)*np.sin(n*np.pi/pib_length*x), 0],
        )
    else:
        y = np.piecewise(
          x,
          [x < pib_lowerlimit, (x >= pib_lowerlimit) & (x <= pib_upperlimit), x > pib_upperlimit],
          [0, lambda x: np.sqrt(2/pib_length)*np.cos(n*np.pi/pib_length*x), 0],
        )
    return y

  # interaction potential between the two particles
  def Vint(x1,x2):
    mg = np.meshgrid(x1,x2)

    ### Gaussian interaction
    # C=30
    # b=0.04
    # y = C*np.exp(-(mg[0]-mg[1])**2/b)

    ### "soft" Coulomb interaction
    # a is the "strength" of the interaction #
    y = 1/(np.abs(mg[0]-mg[1])+a)

    return y

  # unperturbed potential (Particle in a Box)
  def Vpot(x):
      y = np.piecewise(
          x,
          [x < pib_lowerlimit, (x >= pib_lowerlimit) & (x <= pib_upperlimit), x > pib_upperlimit],
          [well_depth, 0, well_depth],
      )
      return y

  # total potential seen by particle 1
  def V1(x1,x2):
      y1 = Vpot(x1)
      y2 = np.trapz(Vint(x1,x2)*psi2**2,x2,dx=2)
      y = y1 + y2
      return y

  # total potential seen by particle 2
  def V2(x1,x2):
      y1 = Vpot(x2)
      y2 = np.trapz(Vint(x1,x2)*psi1**2,x1,dx=1)
      y = y1 + y2
      return y

  # inital guesses for the wave functions
  psi2 = psi_initial(x2,n2)
  psi1 = psi_initial(x1,n1)

  # vectors to save old potentials for deciding when to stop the SCF procedure
  V1_old = np.zeros(len(V1(x1,x2)))
  V2_old = np.zeros(len(V2(x1,x2)))

  #cutoff = 1e-3 # agreement required between current potential and potential from previous iteration to stop SCF
  numiter = 0 # counter

  # the self consistent field loop
  while (np.sum(np.abs(V1(x1,x2)-V1_old)) > cutoff) and (np.sum(np.abs(V2(x1,x2)-V2_old)) > cutoff):
    numiter+=1
    # save old potentials for comparison
    V1_old = V1(x1,x2)
    V2_old = V2(x1,x2)

    # define potential
    def V(x):
      y = np.interp(x, x2, V2(x1,x2))
      return y

    # numerically solve for energies and wave functions of particle 2
    E, E_four = domain.solve(m, V)
    psi = domain.grid(x2, E_four[:,:n_plot])
    psi2 = psi[n2-1,:]

    # define new potential
    def V(x):
      y = np.interp(x, x1, V1(x1,x2))
      return y

    # numerically solve for energies and wave functions of particle 1
    E, E_four = domain.solve(m, V)
    psi = domain.grid(x1, E_four[:,:n_plot])
    psi1 = psi[n1-1,:]

    # print error between current potential and old potential
    print("Iteration %(numiter)d. Potential energy error: %(error)f" % {"numiter":numiter, "error":np.sum(np.abs(V1(x1,x2)-V1_old))})

  # plot initial and final probabilities
  fig = plt.figure()
  plt.xlabel('x (L)')
  plt.ylabel(r'$\psi^2$')
  plt.legend([r'$\psi_1$',r'$\psi_1$(initial)',r'$\psi_2$',r'$\psi_2$(inital)'])
  plt.plot(x1,psi1.flatten()**2,'-k')
  plt.plot(x1,psi_initial(x1,n1)**2,'--k')
  plt.plot(x2,psi2.flatten()**2,'-r')
  plt.plot(x2,psi_initial(x2,n2)**2,'--r')
  plt.show()

In [15]:
cu = False # turn off continous update for smoother operation
style = {'description_width': 'initial'}

# # set up all the sliders to change variables
n1 = widgets.IntSlider(min=1, max=5, step=1, description='n1',continuous_update=cu)
n2 = widgets.IntSlider(min=1, max=5, step=1, description='n2',continuous_update=cu)
a=widgets.Dropdown(options=[('Very Weak', 10), ('Weak', 0.1), ('Medium',0.01), ('Strong',0.001), ('Very strong', 0.0001)],description='Interaction Strength',value=0.1,style=style,continuous_update=cu)
cutoff = widgets.Dropdown(options=[('Tight', 0.0001), ('Medium',0.1), ('Loose',100)],description='Self-Consistency Cutoff',value=0.1,style=style,continuous_update=cu)

# define the output and arrange sliders and plot
out = widgets.interactive_output(run,{'n1':n1,'n2':n2, 'a':a, 'cutoff':cutoff})
controls = widgets.HBox([n1, n2, a, cutoff])
display(widgets.VBox([controls, out]))